In [31]:
################################  Storage_Optimization.ipynb  ####################################
# Author: Sukhendu Sain
# Description: Main file of codebase. Houses main code
# Data: 23-Nov-2024
#################################################################################

In [32]:
# Import Necessary Libraries, Utils, and Config Files
import utils
from config import *
import pandas as pd
import numpy as np
import os, re
import matplotlib.pyplot as plt
# import importlib
# importlib.reload(utils)

### Config - Parameters/Other Variables/File Paths

In [33]:
## Files Path ##
ROOT_FILE_PATH = '\\'.join(os.getcwd().split('\\')[:-1])

AKINS_FOMO_FILE_PATH = os.path.join(ROOT_FILE_PATH, r"Data&Files\AKINS FoMoCo_Piece_Sales_112222_YTD.xlsx")
GPARTS_FILE_PATH = os.path.join(ROOT_FILE_PATH, r"Data&Files\GPARTS Part Measures.xlsx")
WHOLESALE_FILE_PATH = os.path.join(ROOT_FILE_PATH, r"Data&Files\Wholesale JAN_Oct_Parts_Ranking_Counter_Invoices_All_Brands.xlsx")
SERVICE_FILE_PATH = os.path.join(ROOT_FILE_PATH, r"Data&Files\Service JAN_Oct_Parts_Ranking_ROs_All_Brands.xlsx")
COUNTERPAD_FILE_PATH = os.path.join(ROOT_FILE_PATH, r"Data&Files\Counter_Pad_11142024.xlsx")



## Variables
print_df_after_import = True
print_df_data_analyse = True

# Data Import and Clean

In [ ]:
#### Read FILE:: (AKINS FoMoCo_Piece_Sales_112222_YTD.xlsx) into Dataframe
df_Akins = utils.read_excel(AKINS_FOMO_FILE_PATH)
df_Akins['Part#'] = df_Akins['Part#'].apply(lambda a: "".join(str(a).split('-')))
if print_df_after_import: utils.print_df(df_Akins, 200) # Print the Dataframe
# ~1-2secs

In [ ]:
#### Read FILE:: (GPARTS Part Measures.xlsx) into Dataframe
df_Gparts = utils.read_excel(GPARTS_FILE_PATH)
if print_df_after_import: utils.print_df(df_Gparts) # Print the Dataframe
# ~50-60secs

In [ ]:
#### Read FILE:: (Wholesale JAN_Oct_Parts_Ranking_Counter_Invoices_All_Brands.xlsx) into Dataframe
df_Wholesale = utils.read_excel(WHOLESALE_FILE_PATH)

# Clean the Wholesale Dataframe
df_Wholesale = df_Wholesale.drop(columns=[col for col in df_Wholesale.columns if 'Unnamed' in col], inplace=False)
df_Wholesale = df_Wholesale[(df_Wholesale['Vendor'] == 'FOR') | (df_Wholesale['Vendor'] == 'CHR')].reset_index()

if print_df_after_import: utils.print_df(df_Wholesale) # Print the Dataframe
# ~15secs

In [ ]:
#### Read FILE:: (Service JAN_Oct_Parts_Ranking_ROs_All_Brands.xlsx) into Dataframe
df_Service = utils.read_excel(SERVICE_FILE_PATH)

# Clean the Service Dataframe
df_Service = df_Service.drop(columns=[col for col in df_Service.columns if 'Unnamed' in col], inplace=False)
df_Service = df_Service[(df_Service['Vendor'] == 'FOR') | (df_Service['Vendor'] == 'CHR')].reset_index()
df_Service['Qty Sold'] = df_Service['Qty Sold'].astype(int)

if print_df_after_import: utils.print_df(df_Service, 100) # Print the Dataframe
# ~5-6secs

In [38]:
#### Read FILE:: (Counter Pad) into Dataframe

# Data Analysis


### Akins File Analysis

In [ ]:
## Sort 'Sold Pcs' Column Descending
df_Akins = df_Akins.sort_values('Sold Pcs ', ascending=False)
if print_df_data_analyse: utils.print_df(df_Akins, 100) # Print the Dataframe

In [ ]:
## Sum up the 'Sold Pcs' for each unique 'Part Desc.'
part_type_sold_sum = df_Akins.groupby('Description')['Sold Pcs '].sum().reset_index()
part_type_sold_sum.columns = ['Part Desc.', 'Total Sold Pcs.']

# Sort the part_type_sold_sum by Total Sold Pcs. in descending order
part_type_sold_sum = part_type_sold_sum.sort_values('Total Sold Pcs.', ascending=False)

if print_df_data_analyse: utils.print_df(part_type_sold_sum, 5)

In [ ]:
## Visualize above part_type_sold_sum using Bar, One Fig of Top 10 Most Sold Part Types and One for Top 50
top_10 = part_type_sold_sum.head(10)
top_50 = part_type_sold_sum.head(50)

# Create the bar chart for Top 10
plt.figure(figsize=(8, 6))  # Set the figure size
plt.bar(top_10['Part Desc.'], top_10['Total Sold Pcs.'], width=0.5)

# Customize the chart
plt.title('Top 10 Part Types by Total Sold Pieces (Akins)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Sold Pieces', fontsize=16)
plt.xticks(rotation=90)

# Add value labels on top of each bar
for i, v in enumerate(top_10['Total Sold Pcs.']):
    plt.text(i, v, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()


# Create the bar chart for Top 50
plt.figure(figsize=(9, 6))  # Set the figure size
plt.bar(top_50['Part Desc.'], top_50['Total Sold Pcs.'], width=0.5)

# Customize the chart
plt.title('Top 50 Part Types by Total Sold Pieces (Akins)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Sold Pieces', fontsize=16)
plt.xticks(rotation=90)
plt.grid(axis='x', linestyle='--', alpha=1)
plt.grid(axis='y', linestyle='--', alpha=1)

plt.tight_layout()
plt.show()

### GParts File Analysis

In [ ]:
## Analyze 'Active' Column of GParts
total_rows = df_Gparts.shape[0]
active = df_Gparts[df_Gparts["Is Active?"] == 'Yes'].shape[0]
active_percent = ( df_Gparts[df_Gparts["Is Active?"] == 'Yes'].shape[0]/total_rows ) * 100

# Print the Counts/Percentage
print(f"Active Parts: {active}; Active Percentage: {active_percent}%")
print(f"Not Active Parts: {total_rows - active}; Not Active Percentage: {100 - active_percent}%")

# Visualize
labels = ['Active', 'Not Active'] # Create labels and sizes for the pie chart
sizes = [active_percent, 100 - active_percent]

plt.figure(figsize=(10, 8)) # Create the pie chart
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, textprops={'fontsize': 23})

plt.show() # Show the chart

In [ ]:
## Find Rows with 0 in either Dimensions
# Depth (Column Here) = Length (In Docs)

# Count no. of Rows with 0 in either Dimensions; no_all share count with other 3
no_depth = df_Gparts[df_Gparts["Prod Att - Length"] == 0].shape[0]
no_width = df_Gparts[df_Gparts["Prod Att- Width"] == 0].shape[0]
no_height = df_Gparts[df_Gparts["Prod Att - Height"] == 0].shape[0]
no_all = df_Gparts[(df_Gparts["Prod Att - Height"] == 0) & (df_Gparts["Prod Att- Width"] == 0) & (df_Gparts["Prod Att - Length"] == 0)].shape[0]

# Calculate percentages
total_rows = df_Gparts.shape[0]
percent_no_depth = (no_depth / total_rows) * 100
percent_no_width = (no_width / total_rows) * 100
percent_no_height = (no_height / total_rows) * 100
percent_no_all = (no_all / total_rows) * 100

# Count 'Active' Parts having No Dimensions and Dimensions
No_Dim_Active = df_Gparts[(df_Gparts["Prod Att - Height"] == 0) & (df_Gparts["Is Active?"] == 'Yes')].shape[0]
No_Dim_Not_Active = df_Gparts[(df_Gparts["Prod Att - Height"] == 0) & (df_Gparts["Is Active?"] == 'No')].shape[0]
No_Dim_Active_Percent = (No_Dim_Active/no_all) * 100
No_Dim_Not_Active_Percent = (No_Dim_Not_Active/no_all) * 100
With_Dim_Active = df_Gparts[(df_Gparts["Prod Att - Height"] != 0) & (df_Gparts["Is Active?"] == 'Yes')].shape[0]
With_Dim_Not_Active = df_Gparts[(df_Gparts["Prod Att - Height"] != 0) & (df_Gparts["Is Active?"] == 'No')].shape[0]
With_Dim_Active_Percent = (With_Dim_Active/(total_rows - no_all)) * 100
With_Dim_Not_Active_Percent = (With_Dim_Not_Active/(total_rows - no_all)) * 100

# Print the Counts/Percentages
print(f"No Length/Depth: {round(no_depth/total_rows*100, 2)}%; No Width: {round(no_width/total_rows*100, 2)}%; No Height: {round(no_height/total_rows*100, 2)}%; No Dimensions: {round(percent_no_all, 2)}%;")
print(f"Active Parts of No Dimensions: {No_Dim_Active}; Percentage with respect to Parts without Dims: {round(No_Dim_Active_Percent, 2)}%")
print(f"Not Active Parts of No Dimensions: {No_Dim_Not_Active}; Percentage with respect to Parts without Dims: {round(No_Dim_Not_Active_Percent, 2)}%")
print(f"Active Parts with Dimensions: {With_Dim_Active}; Percentage with respect to Parts with Dims: {round(With_Dim_Active_Percent, 2)}%")
print(f"Not Active Parts with Dimensions: {With_Dim_Not_Active}; Percentage with respect to Parts with Dims: {round(With_Dim_Not_Active_Percent, 2)}%")

# Here we find out that a row, if containing 0 in 1 dimension, has 0 in all, or
# A row has either all or none dimensions
# 16% of Rows has 0 Dimensions

In [ ]:
# Visualize
labels = ['Active', 'Not Active'] # Create labels and sizes for the pie chart
sizes = [No_Dim_Active_Percent, No_Dim_Not_Active_Percent]

plt.figure(figsize=(10, 8)) # Create the pie chart
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, textprops={'fontsize': 20})

plt.show() # Show the chart

In [ ]:
## Visualize above No Dimensions Data

# Create the bar graph
plt.figure(figsize=(10, 6)) 
bars = ['Total Rows', 'No Length/Depth', 'No Width', 'No Height']
heights = [100, percent_no_depth, percent_no_width, percent_no_height]

plt.bar(bars, heights)
plt.title(f'Distribution of Rows with No Dimensions', fontsize=16)
plt.ylabel('Percentage of Rows')
plt.xticks(rotation=45, fontsize=16)

# Add labels to each bar
for i, v in enumerate(heights):
    plt.text(i, v, str(round(v, 2)) + '%', ha='center', va='bottom')

# Show the legend and display the chart
plt.tight_layout()
plt.show()

In [ ]:
## Visualize everything Analyzed above in a single Stacked Bar Chart

categories = ['Total Rows', 'Dimensional Rows', 'No Dimensional Rows']
values1 = [active, With_Dim_Active, No_Dim_Active]
values2 = [total_rows - active, With_Dim_Not_Active, No_Dim_Not_Active]

# Create figure and axis objects
fig, ax = plt.subplots(figsize=(8, 6))

# Plot the stacked bar chart
ax.bar(categories, values1, label='Active', width = 0.3)
ax.bar(categories, values2, bottom=values1, label='Not Active', width = 0.3)

for i, v in enumerate([total_rows, With_Dim_Active+With_Dim_Not_Active, No_Dim_Active+No_Dim_Not_Active]):
    plt.text(i, v, f'Total: {str(v)}', ha='center', va='bottom')

# Customize the plot
ax.set_ylabel('Values')
ax.set_title('Bar Chart of Active and Non-Active Parts (GParts)')
ax.legend()

ax.set_yticks(np.linspace(0, 231045, 20))  # Set more ticks on y-axis

plt.grid(axis='x', linestyle='--', alpha=1)
plt.grid(axis='y', linestyle='--', alpha=1)

# Display the chart
plt.tight_layout()
plt.show()

### Wholesale Files Analysis

In [ ]:
## Sort 'Sold' Column Descending
df_Wholesale = df_Wholesale.sort_values('Sold', ascending=False)
if print_df_data_analyse: utils.print_df(df_Wholesale, 100) # Print the Dataframe

In [ ]:
## Count the number of negative sold pcs.
neg_sold_count = df_Wholesale[df_Wholesale['Sold'] < 0].shape[0]
print(f"Number of Negative Sold Values: {neg_sold_count}")

In [ ]:
## Sum up the 'Gross Profit' for each unique 'Part Desc.'
part_type_profit_sum = df_Wholesale.groupby('Description')['Gross Profit'].sum().reset_index()
part_type_profit_sum.columns = ['Part Desc.', 'Total Gross Profit']
part_type_profit_sum['Total Gross Profit'] = pd.to_numeric(part_type_profit_sum['Total Gross Profit'], errors='coerce').round()

# Sort the part_type_sold_sum by Total Sold Pcs. in descending order
part_type_profit_sum = part_type_profit_sum.sort_values('Total Gross Profit', ascending=False)
part_type_profit_sum['Total Gross Profit'] = part_type_profit_sum['Total Gross Profit'].round(-1)

if print_df_data_analyse: utils.print_df(part_type_profit_sum, 5)

# Visualize using Bar of Top 10
top_10 = part_type_profit_sum.head(10)

# Create the bar chart for Top 10
plt.figure(figsize=(8, 6))  # Set the figure size
plt.bar(top_10['Part Desc.'], top_10['Total Gross Profit'], width=0.5)

# Customize the chart
plt.title('Top 10 Part Types by Total Gross Profit (Wholesale)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Gross Profit', fontsize=16)
plt.xticks(rotation=90)

# Add value labels on top of each bar
for i, v in enumerate(top_10['Total Gross Profit']):
    plt.text(i, v, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
## Sum up the 'Sold' for each unique 'Part Desc.'
part_type_sold_sum = df_Wholesale.groupby('Description')['Sold'].sum().reset_index()
part_type_sold_sum.columns = ['Part Desc.', 'Total Sold Pcs.']

# Sort the part_type_sold_sum by Total Sold Pcs. in descending order
part_type_sold_sum = part_type_sold_sum.sort_values('Total Sold Pcs.', ascending=False)

if print_df_data_analyse: utils.print_df(part_type_sold_sum, 5)

In [ ]:
## Visualize above part_type_sold_sum using Bar, One Fig of Top 10 Most Sold Part Types and One for Top 50
top_10 = part_type_sold_sum.head(10)
top_50 = part_type_sold_sum.head(50)

# Create the bar chart for Top 10
plt.figure(figsize=(8, 6))  # Set the figure size
plt.bar(top_10['Part Desc.'], top_10['Total Sold Pcs.'], width=0.5)

# Customize the chart
plt.title('Top 10 Part Types by Total Sold Pieces (Wholesale)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Sold Pieces', fontsize=16)
plt.xticks(rotation=90)

# Add value labels on top of each bar
for i, v in enumerate(top_10['Total Sold Pcs.']):
    plt.text(i, v, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()


# Create the bar chart for Top 50
plt.figure(figsize=(9, 6))  # Set the figure size
plt.bar(top_50['Part Desc.'], top_50['Total Sold Pcs.'], width=0.5)

# Customize the chart
plt.title('Top 50 Part Types by Total Sold Pieces (Wholesale)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Sold Pieces', fontsize=16)
plt.xticks(rotation=90)
plt.grid(axis='x', linestyle='--', alpha=1)
plt.grid(axis='y', linestyle='--', alpha=1)

plt.tight_layout()
plt.show()

### Service File Analysis

In [ ]:
## Sort 'Qty Sold' Column Descending
df_Service = df_Service.sort_values('Qty Sold', ascending=False)
if print_df_data_analyse: utils.print_df(df_Service, 100) # Print the Dataframe

In [ ]:
## Count the number of negative sold pcs.
neg_sold_count = df_Service[df_Service['Qty Sold'].astype(int) < 0].shape[0]
print(f"Number of Negative Sold Values: {neg_sold_count}")

In [ ]:
## Sum up the 'Gross Profit' for each unique 'Part Desc.'
part_type_profit_sum = df_Service.groupby('Description')['Gross Profit'].sum().reset_index()
part_type_profit_sum.columns = ['Part Desc.', 'Total Gross Profit']
part_type_profit_sum['Total Gross Profit'] = pd.to_numeric(part_type_profit_sum['Total Gross Profit'], errors='coerce').round()

# Sort the part_type_sold_sum by Total Sold Pcs. in descending order
part_type_profit_sum = part_type_profit_sum.sort_values('Total Gross Profit', ascending=False)
part_type_profit_sum['Total Gross Profit'] = part_type_profit_sum['Total Gross Profit'].round(-1)

if print_df_data_analyse: utils.print_df(part_type_profit_sum, 5)

# Visualize using Bar of Top 10
top_10 = part_type_profit_sum.head(10)

# Create the bar chart for Top 10
plt.figure(figsize=(8, 6))  # Set the figure size
plt.bar(top_10['Part Desc.'], top_10['Total Gross Profit'], width=0.5)

# Customize the chart
plt.title('Top 10 Part Types by Total Gross Profit (Service)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Gross Profit', fontsize=16)
plt.xticks(rotation=90)

# Add value labels on top of each bar
for i, v in enumerate(top_10['Total Gross Profit']):
    plt.text(i, v, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
## Sum up the 'Sold' for each unique 'Part Desc.'
part_type_sold_sum = df_Service.groupby('Description')['Qty Sold'].sum().reset_index()
part_type_sold_sum.columns = ['Part Desc.', 'Total Sold Pcs.']

# Sort the part_type_sold_sum by Total Sold Pcs. in descending order
part_type_sold_sum = part_type_sold_sum.sort_values('Total Sold Pcs.', ascending=False)

if print_df_data_analyse: utils.print_df(part_type_sold_sum, 5)

In [ ]:
## Visualize above part_type_sold_sum using Bar, One Fig of Top 10 Most Sold Part Types and One for Top 50
top_10 = part_type_sold_sum.head(10)
top_50 = part_type_sold_sum.head(50)

# Create the bar chart for Top 10
plt.figure(figsize=(8, 6))  # Set the figure size
plt.bar(top_10['Part Desc.'], top_10['Total Sold Pcs.'], width=0.5)

# Customize the chart
plt.title('Top 10 Part Types by Total Sold Pieces (Service)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Sold Pieces', fontsize=16)
plt.xticks(rotation=90)

# Add value labels on top of each bar
for i, v in enumerate(top_10['Total Sold Pcs.']):
    plt.text(i, v, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()


# Create the bar chart for Top 50
plt.figure(figsize=(9, 6))  # Set the figure size
plt.bar(top_50['Part Desc.'], top_50['Total Sold Pcs.'], width=0.5)

# Customize the chart
plt.title('Top 50 Part Types by Total Sold Pieces (Service)', fontsize=20)
plt.xlabel('Part Description', fontsize=16)
plt.ylabel('Total Sold Pieces', fontsize=16)
plt.xticks(rotation=90)
plt.grid(axis='x', linestyle='--', alpha=1)
plt.grid(axis='y', linestyle='--', alpha=1)

plt.tight_layout()
plt.show()

In [ ]:
## Find Number of Matching Part Numbers in each of the Dataframe

# The Dataframes to match
all_dfs = [df_Akins, df_Gparts]

# Part# Column Name of each DF
part_number_columns = ['Part#', 'Svc Part Number']

# Find common part numbers
common_part_numbers = set(all_dfs[0][part_number_columns[0]])
for i in range(1, len(all_dfs)):
    common_part_numbers &= set(all_dfs[i][part_number_columns[i]])

print(f"Part numbers common to all DataFrames: {len(common_part_numbers)}")
#if print_df_data_analyse: utils.print_df(df_Wholesale[df_Wholesale["Part Number"].isin(common_part_numbers)], 100)

# Data Processing & Calculation

In [59]:
## Make a Big Final Dataframe
# It will have the Columns - 'Part Number', 'Part Type', 'Active', 'Sold (Pcs.)', 'Length/Depth', 'Width', 'Height', 'Zone', 'Storage Type', 'Sub Storage', 'Number of Storage needed'
# It will have all the rows with common part nos. from all 4 Files, having Appropriate Sold Pcs. Values, and Dimensions

main_list = []

gParts_PartNos = set(df_Gparts['Svc Part Number'])

common_part_numbers = gParts_PartNos & set(df_Akins['Part#'])
for pn, pt, ac, s, ld, w, h in zip(common_part_numbers, df_Gparts["Svc Part Number Description"], df_Gparts['Is Active?'], df_Akins['Sold Pcs '], df_Gparts['Prod Att - Length'], df_Gparts['Prod Att- Width'], df_Gparts['Prod Att - Height']):
    main_list.append([pn, pt, ac, s, ld, w, h, "", "", "", ""])

common_part_numbers = gParts_PartNos & set(df_Wholesale['Part Number'])
for pn, pt, ac, s, ld, w, h in zip(common_part_numbers, df_Gparts["Svc Part Number Description"], df_Gparts['Is Active?'], df_Wholesale['Sold'], df_Gparts['Prod Att - Length'], df_Gparts['Prod Att- Width'], df_Gparts['Prod Att - Height']):
    main_list.append([pn, pt, ac, s, ld, w, h, "", "", "", ""])


common_part_numbers = gParts_PartNos & set(df_Service['* indicates a superseded part\nPart Number'])
for pn, pt, ac, s, ld, w, h in zip(common_part_numbers, df_Gparts["Svc Part Number Description"], df_Gparts['Is Active?'], df_Service['Qty Sold'], df_Gparts['Prod Att - Length'], df_Gparts['Prod Att- Width'], df_Gparts['Prod Att - Height']):
    main_list.append([pn, pt, ac, s, ld, w, h, "", "", "", ""])

df_Main = pd.DataFrame(main_list)
df_Main.columns = ['Part#', 'Part Type', 'Sold', 'Depth', 'Width', 'Height', 'Zone', 'StorageType', 'SubStorage', 'Num. Storage Required']

utils.print_df(df_Main)

ValueError: Length mismatch: Expected axis has 11 elements, new values have 10 elements

In [ ]:
data = []
sum = 0
totalSoldPCs = int(df_Wholesale["Total Sold"].sum())
print(sum, totalSoldPCs)
for i in range(df_Wholesale.shape[0]):
    zone = ""
    if sum/totalSoldPCs <= 0.2:
        zone = "Red Hot Zone"
    if sum/totalSoldPCs > 0.2 and sum/totalSoldPCs <= 0.4:
        zone = "Orange Zone"
    if sum/totalSoldPCs > 0.4 and sum/totalSoldPCs <= 0.6:
        zone = "Yellow Zone"
    if sum/totalSoldPCs > 0.6 and sum/totalSoldPCs <= 0.8:
        zone = "Green Zone"
    if sum/totalSoldPCs > 0.8:
        zone = "Blue Zone"
    data.append([df_Wholesale["Part Number"].iloc[i], zone, sum])
    sum = sum + df_Wholesale["Total Sold"].iloc[i]
df_zones = pd.DataFrame(data)
utils.print_df(df_zones,None)

with open("htt.txt", "w") as f:
    f.write(df_zones.to_string())